<center>
    <h1 style="color:deeppink">Just between us SQL Friends... 👑</h1>
    <b style="color:deeppink">RuPaul's Drag Race: Visualized<b>
    <img src="media/header.gif" height="100px">
</center>

<h2 style="color:deeppink">Import Libraries & Connect SQLite Database</h2>

In [140]:
import pandas as pd
import sqlite3

drag_race_data_base = sqlite3.connect("drag_race_data.db")

<h2 style="color:deeppink">Define Constants, Global Variables, and API values</h2>

In [218]:
#API URL
DRAG_RACE_API = "http://www.nokeynoshade.party/api/"

#API Endpoints
SEASONS = "seasons/{}"
QUEENS = "queens/all"
QUEENS_BY_SEASON = "seasons/{}/queens"
EPISODES = "episodes"
EPISODES_BY_SEASON = "seasons/{}/episodes"
CHALLENGES_BY_SEASON = "seasons/{}/challenges"

<h2 style="color:deeppink">Define Functions</h2>

In [212]:
#pull data from the API by passing the desired endpoint and 
#     the optional SEASON argument for the endpoints that require it
#     returns a pandas dataframe
def get_data(endpoint, SEASON = None):
    data = pd.read_json(DRAG_RACE_API + endpoint.format(SEASON))
    return data

<h2 style="color:deeppink">Define SQLite Queries</h2>

<h2 style="color:deeppink">Sandbox</h2>
<b style="color:deeppink">Use this space to experiment. Nothing below will be included in the finished product.</b>

In [254]:
get_data(SEASONS, 7)

,id,seasonNumber,winnerId,image_url,queens
0,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 70, 'name': 'April Carrion', 'place': 11}"
1,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 74, 'name': 'Trinity K Bonet', 'place': 7}"
2,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 73, 'name': 'Laganja Estranja', 'place'..."
3,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 76, 'name': 'Joslyn Fox', 'place': 6}"
4,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 65, 'name': 'Magnolia Crawford', 'place..."
5,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 66, 'name': 'Vivacious', 'place': 12}"
6,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 71, 'name': 'Gia Gunn', 'place': 10}"
7,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 64, 'name': 'Kelly Mantle', 'place': 14}"
8,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 72, 'name': 'Milk', 'place': 9}"
9,7,6,80,https://vignette3.wikia.nocookie.net/logosrupa...,"{'id': 78, 'name': 'Courtney Act', 'place': 3}"
